In [10]:
import pandas as pd
import numpy as np


def get_student_df(student_info_df, include_transfer=False, include_nan=False, term_cut="FA 2022", include_covid = True, term_start="FA 2007", is_graduation=False):
    # Extract term codes
    term_cut_cd = student_info_df.loc[student_info_df['FIRST_TERM_ATTND_SHORT_DES'] == term_cut, 'FIRST_TERM_ATTND_CD'].iloc[0]
    term_start_cd = student_info_df.loc[student_info_df['FIRST_TERM_ATTND_SHORT_DES'] == term_start, 'FIRST_TERM_ATTND_CD'].iloc[0]

    # Apply combined filtering
    student_info_df = student_info_df[
        (student_info_df['FIRST_TERM_ATTND_CD'] < term_cut_cd) &
        (student_info_df['FIRST_TERM_ATTND_CD'] > term_start_cd) &
        (student_info_df['STDNT_SEX_SHORT_DES'] != 'Unknown')
    ]

    # Filter out COVID dates if necessary
    if not include_covid and not is_graduation:
        covid_begin_cd = student_info_df.loc[student_info_df['FIRST_TERM_ATTND_SHORT_DES'] == 'WN 2020', 'FIRST_TERM_ATTND_CD'].iloc[0]
        covid_end_cd = student_info_df.loc[student_info_df['FIRST_TERM_ATTND_SHORT_DES'] == 'WN 2022', 'FIRST_TERM_ATTND_CD'].iloc[0]
        student_info_df = student_info_df[
            (student_info_df['FIRST_TERM_ATTND_CD'] < covid_begin_cd) |
            (student_info_df['FIRST_TERM_ATTND_CD'] > covid_end_cd)
        ]

    # Prepare entry type filter
    first_ug_type = ['First-Year', 'First-Year Assumed']
    if include_transfer:
        first_ug_type.extend(['Transfer', 'Transfer Assumed'])

    # Apply the final filter based on `include_nan`
    if include_nan:
        match_student_df = student_info_df.loc[
            (student_info_df['FIRST_UG_ENTRY_TYP_DES'].isin(first_ug_type)) |
            (student_info_df['FIRST_UG_ENTRY_TYP_DES'].isna())
        ]
    else:
        match_student_df = student_info_df.loc[
            student_info_df['FIRST_UG_ENTRY_TYP_DES'].isin(first_ug_type)
        ]

    # print(len(match_student_df))
    return match_student_df   


# Calculate Age When they first enroll
def add_age_col(relevant_student_df):
    relevant_student_df.loc[:,'Begin_Year'] = relevant_student_df['FIRST_TERM_ATTND_BEGIN_YR_MO'].astype(str).str[0:4]
    
    # Convert to integer, but safely turning errors into NaNs
    relevant_student_df.loc[:,'Begin_Year'] = pd.to_numeric(relevant_student_df['Begin_Year'], errors='coerce')

    # Calculate 'STARTING_AGE', handling NaN values to avoid errors from subtracting NaNs
    relevant_student_df.loc[:,'STARTING_AGE'] = relevant_student_df.apply(
        lambda row: row['Begin_Year'] - row['STDNT_BIRTH_YR'] 
                    if pd.notna(row['Begin_Year']) and pd.notna(row['STDNT_BIRTH_YR']) and (row['Begin_Year'] - row['STDNT_BIRTH_YR'] > 0)
                    else np.nan,
        axis=1
    )
    
    return relevant_student_df.drop(columns=['Begin_Year'])

# Add the first gen indicator column   
def add_first_gen_col(relevant_student_df, allow_nan=False):
    
    def categorize_first_gen(parent_edu):
        if parent_edu in ['High School diploma', 'Elementary School only', 'Less than High School']:
            return 1
        elif allow_nan and (pd.isna(parent_edu) or parent_edu in ["Don't Know", " ", "Not Indicated"]):
            return np.nan
        else:
            return 0
    relevant_student_df['IS_FIRST_GEN'] = relevant_student_df['PRNT_MAX_ED_LVL_DES'].apply(categorize_first_gen)
    
    return relevant_student_df

# Add number of accepted transfer credits when student start
def add_transfer_credits_col(relevant_student_df, transfer_info_df):
    # relevant_student_df = relevant_student_df.copy()
    # transfer_info_df = transfer_info_df.copy()
    relevant_student_df['FIRST_TERM_ATTND_CD'] = relevant_student_df['FIRST_TERM_ATTND_CD'].astype(str)
    transfer_info_df['ARTCLT_TERM_CD'] = transfer_info_df['ARTCLT_TERM_CD'].astype(str)
    merged_df = pd.merge(relevant_student_df, transfer_info_df, left_on=['STDNT_ID', 'FIRST_TERM_ATTND_CD'], right_on=['STDNT_ID', 'ARTCLT_TERM_CD'], how='left')

    # Select the 'UNITS_TRNSFR_NBR' column from the merged DataFrame to create the 'TRANSFER_CREDITS' column
    relevant_student_df['TRANSFER_CREDITS'] = merged_df['UNITS_TRNSFR_NBR']

    return relevant_student_df

# Add a column of aggregated SAT score
def add_max_sat_col(relevant_student_df):
    
    # Convert ACT to SAT
    act_to_sat_map = {
    36: 1590, 35: 1540, 34: 1500, 33: 1460, 32: 1430, 31: 1400,
    30: 1370, 29: 1340, 28: 1310, 27: 1280, 26: 1240, 25: 1210,
    24: 1180, 23: 1140, 22: 1110, 21: 1080, 20: 1040, 19: 1010,
    18: 970, 17: 930, 16: 890, 15: 850, 14: 800, 13: 760,
    12: 710, 11: 670, 10: 630, 9: 590
    }
    # relevant_student_df = relevant_student_df.copy()
    relevant_student_df['ACT_TO_SAT'] = relevant_student_df['MAX_ACT_COMP_SCR'].map(act_to_sat_map)
    
    relevant_student_df['MAX_SAT_SCR'] = relevant_student_df[['ACT_TO_SAT', 'MAX_SATI_TOTAL_CALC_SCR', 'MAX_SATI_TOTAL_MSS_ERWS_SCR']].max(axis=1)
    relevant_student_df['MAX_SAT_SCR'] = relevant_student_df['MAX_SAT_SCR'].replace(0, np.nan)
    
    return relevant_student_df.drop(columns=['ACT_TO_SAT'])


def fill_income_col(relevant_student_df, income_choice='zip_mode'):
    """
    income_choice: "mode" (mode of everything except NaN) / "zip_mode" (mode of the zip code except NaN) / "nan" (leave n)
    """
    # relevant_student_df = relevant_student_df.copy()
    relevant_student_df['EST_GROSS_FAM_INC_DES'] = relevant_student_df['EST_GROSS_FAM_INC_DES'].replace(' ', np.nan)
    if income_choice == 'mode':
        overall_mode = relevant_student_df['EST_GROSS_FAM_INC_DES'].mode(dropna=True)[0]
        relevant_student_df['EST_GROSS_FAM_INC_DES'] = relevant_student_df['EST_GROSS_FAM_INC_DES'].fillna(overall_mode)
    elif income_choice == 'zip_mode':
        group_by_zip = relevant_student_df.groupby('FIRST_US_PRMNNT_RES_PSTL_5_CD')['EST_GROSS_FAM_INC_DES'].transform(lambda x: x.mode(dropna=True)[0] if not x.mode().empty else np.nan)
        relevant_student_df['EST_GROSS_FAM_INC_DES'] = relevant_student_df['EST_GROSS_FAM_INC_DES'].fillna(group_by_zip)
    return relevant_student_df
        
    
    
 
    

In [ ]:
# add_features_df = pd.read_csv('add_features.csv')
# print(add_features_df['STDNT_SEX_SHORT_DES'].unique())

In [3]:
# Add the column of cumulative GPA right before next Fall
def add_cum_gpa_col(relevant_student_df, term_info_df):
    # Group by Student ID to process each student's term records
    def get_relevant_gpa(group):
        last_gpa = None  # To store the last GPA before a Fall term
        # Iterate through sorted group assuming it is sorted by TERM_SHORT_DES
        for idx, row in group.iterrows():
            # Check if the term is a Fall term
            if row['TERM_SHORT_DES'].startswith('FA'):
                if last_gpa is not None:
                    # Return the last GPA if there's one before the Fall term
                    return last_gpa
            last_gpa = row['CUM_GPA']
        # If there's no Fall term or it's the last term, return the last available GPA
        return last_gpa

    # Apply the function to each student's group
    gpas = term_info_df.groupby('STDNT_ID').apply(get_relevant_gpa)
    gpas.name = 'FIRST_YR_CUM_GPA'
    
    # Merge the GPA data back into the relevant student DataFrame
    # Ensure that both dataframes have 'STDNT_ID' as a common key
    relevant_student_df = relevant_student_df.merge(gpas, on='STDNT_ID', how='left')

    return relevant_student_df
    

In [4]:
# Add column of the number of credits taken in the first year
def add_credits_taken_col(relevant_student_df, term_info_df):
    def get_relevant_credits(group):
        taken_credits = 0
        for idx, row in group.iterrows():
            if row['TERM_SHORT_DES'].startswith('FA'):
                if taken_credits != 0:
                    return taken_credits
            taken_credits += row['UNIT_TAKEN_NBR']
        return taken_credits
    
    credits = term_info_df.groupby('STDNT_ID').apply(get_relevant_credits)
    credits.name = 'FIRST_YR_TAKEN_CREDITS'
    relevant_student_df = relevant_student_df.merge(credits, on='STDNT_ID', how='left')
    return relevant_student_df
      

In [5]:
# Add column of the major
def add_major_col(relevant_student_df, term_info_df):
    # relevant_student_df = relevant_student_df.copy()
    term_info_df = term_info_df.copy()
    relevant_student_df['FIRST_TERM_ATTND_CD'] = relevant_student_df['FIRST_TERM_ATTND_CD'].astype(str)
    term_info_df['TERM_CD'] = term_info_df['TERM_CD'].astype(str)
    merged_df = pd.merge(relevant_student_df, term_info_df, left_on=['STDNT_ID', 'FIRST_TERM_ATTND_CD'], right_on=['STDNT_ID', 'TERM_CD'], how='left')

    relevant_student_df['PGM_1_MAJOR_1_CIP_DES'] = merged_df['PGM_1_MAJOR_1_CIP_DES']

    return relevant_student_df

In [6]:
# Premature Featurea and Target
def premature(student_info_df, term_info, include_transfer, include_covid):
    relevant_student_df = get_student_df(student_info_df, include_transfer=include_transfer, include_covid=include_covid)
    reenroll_target = generate_reenroll_target(relevant_student_df, term_info_df)
    # add_first_gen_df = add_first_gen_col(relevant_student_df)
    # add_age_df = add_age_col(add_first_gen_df)
    # add_transfer_credit_df = add_transfer_credits_col(add_age_df, transfer_info_df)
    # add_max_sat_df = add_max_sat_col(add_transfer_credit_df)
    # add_major_df = add_major_col(add_max_sat_df, term_info_df)
    # add_cum_gpa_df = add_cum_gpa_col(add_major_df, term_info_df)
    # add_credits_taken_df = add_credits_taken_col(add_cum_gpa_df, term_info_df)
    # print(len(reenroll_target[reenroll_target==1]), len(reenroll_target[reenroll_target==0]))
    return relevant_student_df, reenroll_target

def graduation_premature(student_info_df, include_transfer):
    relevant_student_df = get_student_df(student_info_df, include_transfer=include_transfer, include_covid=False, term_cut = "FA 2018", term_start = "FA 2003", is_graduation=True)
    graduation_df = generate_graduation_target(relevant_student_df)
    # print(len(graduation_df[graduation_df['GRADUATED_IN_4'] == 1]), len(graduation_df[graduation_df['GRADUATED_IN_4'] == 0]), len(graduation_df[graduation_df['GRADUATED_IN_6'] == 1]), len(graduation_df[graduation_df['GRADUATED_IN_6'] == 0]))
    return relevant_student_df, graduation_df

In [7]:
# Get Features DF
def generate_features_df(relevant_student_df, include_race=True, include_sex=True, include_sat_act=True, income_choice='zip_mode'):
    """
    
    Include Race: True / False
    
    Include Sex: True / False
    
    income_choice: "mode" (mode of everything except NaN) / "zip_mode" (mode of the zip code except NaN) / "nan" (leave n)
    
    
    """
    feature_columns = ['STDNT_INTL_IND', 'STDNT_NTV_ENG_SPKR_IND', 'FIRST_US_PRMNNT_RES_PSTL_5_CD', 'PRNT_MAX_ED_LVL_DES', 'FIRST_TERM_ATTND_SHORT_DES',
                      'STARTING_AGE', 'IS_FIRST_GEN', 'TRANSFER_CREDITS', 'MAX_SAT_SCR', 'EST_GROSS_FAM_INC_DES', 'FIRST_YR_CUM_GPA', 
                      'FIRST_YR_TAKEN_CREDITS', 'HS_GPA', 'HS_CALC_IND', 'HS_CHEM_LAB_IND', 'ACAD_LVL_BOT_SHORT_DES', 'PGM_1_MAJOR_1_CIP_DES', 'FIRST_UG_ENTRY_TYP_DES']

In [8]:
# Get if re-enroll next-year target
def generate_reenroll_target(relevant_student_df, term_info_df):
    
    # relevant_student_df = relevant_student_df.copy()
    # term_info_df = term_info_df.copy()
    
    # Prepare the DataFrame to handle missing and irregular term descriptions
    relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'] = relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'].fillna('Unknown')

    # Split the term descriptions safely
    terms = relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'].str.split()

    # Extract terms and years safely, handle cases where split might not produce two parts
    relevant_student_df['First_Term'] = terms.apply(lambda x: x[0] if len(x) > 0 else 'Unknown')
    relevant_student_df['First_Year'] = terms.apply(lambda x: int(x[1]) if len(x) > 1 and x[1].isdigit() else 'Unknown')

    # Determine the target term based on the first attended term
    def get_target_term(row):
        if row['First_Year'] == 'Unknown':
            return 'Unknown'  # Can't compute target term without a year
        if row['First_Term'] == 'FA':
            return f'FA {row['First_Year'] + 1}'
        return f'FA {row['First_Year']}'

    relevant_student_df['Target_Term'] = relevant_student_df.apply(get_target_term, axis=1)
    
    term_info_df['CLEANED_TERM_SHORT_DES'] = term_info_df['TERM_SHORT_DES'].str.extract(r'([A-Z]{2} \d{4})')  # Extracts format like "FA 1999"

    # Create a composite key to check the existence of the target term in term_info_df
    relevant_student_df['student_term_key'] = relevant_student_df['STDNT_ID'].astype(str) + '_' + relevant_student_df['Target_Term']
    term_info_df['student_term_key'] = term_info_df['STDNT_ID'].astype(str) + '_' + term_info_df["CLEANED_TERM_SHORT_DES"]

    # Check if the target term exists for each student
    existing_terms = set(term_info_df['student_term_key'])
    relevant_student_df['IS_REENROLL'] = relevant_student_df['student_term_key'].isin(existing_terms).astype(int)

    # # Clean up the DataFrame by removing temporary columns
    result = relevant_student_df['IS_REENROLL']
    relevant_student_df.drop(['First_Term', 'First_Year', 'student_term_key', 'Target_Term', 'IS_REENROLL'], axis=1, inplace=True)

    return result

In [9]:
# Get if graduate in 4 years or 6 years
def generate_graduation_target(relevant_student_df):
   
    # Prepare the DataFrame to handle missing and irregular term descriptions
    relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'] = relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'].fillna('Unknown')

    # Split the term descriptions safely
    terms = relevant_student_df['FIRST_TERM_ATTND_SHORT_DES'].str.split()

    # Extract terms and years safely, handle cases where split might not produce two parts
    relevant_student_df['First_Year'] = terms.apply(lambda x: int(x[1]) if len(x) > 1 and x[1].isdigit() else 'Unknown')

    relevant_student_df['UM_DGR_1_CMPLTN_TERM_DES'] = relevant_student_df['UM_DGR_1_CMPLTN_TERM_DES'].fillna('Unknown')
    graduation_term = relevant_student_df['UM_DGR_1_CMPLTN_TERM_DES'].str.split()
    
    relevant_student_df['Graduation_Year'] = graduation_term.apply(lambda x: int(x[1]) if len(x) > 1 and x[1].isdigit() else 'Unknown')
    
    # Calculate whether the student graduated in 4 or 6 years, handling 'Unknown' cases
    relevant_student_df['GRADUATED_IN_4'] = relevant_student_df.apply(
        lambda row: 1 if row['Graduation_Year'] != 'Unknown' and row['First_Year'] != 'Unknown' and (row['Graduation_Year'] - row['First_Year'] <= 4) else 0,
        axis=1
    )

    relevant_student_df['GRADUATED_IN_6'] = relevant_student_df.apply(
        lambda row: 1 if row['Graduation_Year'] != 'Unknown' and row['First_Year'] != 'Unknown' and (row['Graduation_Year'] - row['First_Year'] <= 6) else 0,
        axis=1
    )

    # Clean up the DataFrame by removing temporary columns
    result = relevant_student_df[['GRADUATED_IN_4', 'GRADUATED_IN_6']]
    
    relevant_student_df.drop(['First_Year', 'Graduation_Year', 'GRADUATED_IN_6', 'GRADUATED_IN_4'], axis=1, inplace=True)
    return result